# Attention : 
# Faire "File -> Save a copy in Drive" avant de commencer à modifier le notebook, sinon vos modifications ne seront pas sauvegardées.


Avant de commencer le TP, vérifiez que vous êtes sur un environnement GPU et python 3 : 

Exécution -> Modifier le type d'exécution -> Type d'exécution = python2, Accélerateur matériel = GPU

In [1]:
import argparse
import os
import time

import PIL
from PIL import Image

import numpy as np
import torchvision
import pickle

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.utils.data
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

# Partie 1 : Architecture VGG16

In [6]:
!wget https://github.com/cdancette/deep-learning-polytech-tp6-7/raw/master/tp8/imagenet_classes.pkl

# Bonus : Classifiez des exemples avec vgg16 et commentez le résultat dans votre rapport.
!wget --content-disposition https://unsplash.com/photos/gKXKBY-C-Dk/download?force=true -O cat.jpg
!wget --content-disposition https://unsplash.com/photos/qO-PIF84Vxg/download?force=true -O dog.jpg

--2021-01-27 14:11:18--  https://github.com/cdancette/deep-learning-polytech-tp6-7/raw/master/tp8/imagenet_classes.pkl
Résolution de github.com (github.com)… 140.82.121.4
Connexion à github.com (github.com)|140.82.121.4|:443… connecté.
requête HTTP transmise, en attente de la réponse… 302 Found
Emplacement : https://raw.githubusercontent.com/cdancette/deep-learning-polytech-tp6-7/master/tp8/imagenet_classes.pkl [suivant]
--2021-01-27 14:11:21--  https://raw.githubusercontent.com/cdancette/deep-learning-polytech-tp6-7/master/tp8/imagenet_classes.pkl
Résolution de raw.githubusercontent.com (raw.githubusercontent.com)… 151.101.120.133
Connexion à raw.githubusercontent.com (raw.githubusercontent.com)|151.101.120.133|:443… connecté.
requête HTTP transmise, en attente de la réponse… 200 OK
Taille : 35454 (35K) [text/plain]
Enregistre : «imagenet_classes.pkl.1»

imagenet_classes.pk 100%[===================>]  34,62K  --.-KB/s    ds 0,08s   

2021-01-27 14:11:22 (426 KB/s) - «imagenet_classes.

In [ ]:
torchvision.models.vgg.model_urls["vgg16"] = "http://webia.lip6.fr/~robert/cours/rdfia/vgg16-397923af.pth"
os.environ["TORCH_HOME"] = "/tmp/torch"
vgg16 = torchvision.models.vgg16(pretrained=True)
vgg16.eval() 
imagenet_classes = pickle.load(open('imagenet_classes.pkl', 'rb'))
img = PIL.Image.open("dog.jpg")
img = img.resize((224, 224), PIL.Image.BILINEAR)
img = np.array(img, dtype=np.float32) / 255
img = img.transpose((2, 0, 1))

img = np.expand_dims(img, 0) # transformer en batch contenant une image
x = torch.Tensor(img)

y = vgg16.forward(x)
y = y.detach() # transformation en array numpy
y=y.numpy()
# TODO récupérer la classe prédite et son score de confiance
print(type(imagenet_classes))
print("la classe prédite est ",imagenet_classes[np.argmax(y)])
print('le score de confiance est',np.max(y))
print(vgg16.features[0](x).shape)

plt.imshow(vgg16.features[0](x)[0][0].detach())
plt.show()

Downloading: "http://webia.lip6.fr/~robert/cours/rdfia/vgg16-397923af.pth" to /tmp/torch/hub/checkpoints/vgg16-397923af.pth


# Partie 2: Transfer Learning avec VGG16 sur 15 Scene

In [ ]:
!wget https://github.com/cdancette/deep-learning-polytech-tp6-7/raw/master/tp8/15ScenesData.zip
!unzip 15ScenesData.zip

In [ ]:
ls 15SceneData/test/bedroom/

In [ ]:
class VGG16relu7(torch.nn.Module):
    def __init__(self,vgg16):
        super(VGG16relu7,self).__init__()
        self.features = torch.nn.Sequential(*list(vgg16.features.children()))
        self.classifier = torch.nn.Sequential(*list(vgg16.classifier.children())[:-2])


    def forward(self,x):
        x = self.features(x)
        x = x.view(x.size(0),-1)
        x = self.classifier(x)
        return x 

In [ ]:

PRINT_INTERVAL = 50
CUDA = True

def get_dataset(batch_size, path):

    def duplicateChannel(img):
        # Cette fonction permet de recopier 3 fois une image qui
        # ne serait que sur 1 channel (donc image niveau de gris)
        # pour la "transformer" en image RGB. Utilisez la avec
        # transform.Lambda
        img = img.convert('L')
        np_img = np.array(img, dtype=np.uint8)
        np_img = np.dstack([np_img, np_img, np_img])
        img = Image.fromarray(np_img, 'RGB')
        return img

    mu,sigma = [0.485,0.456,0.406],[0.229,0.224,0.225]
    train_dataset = datasets.ImageFolder(path+'/train',
        transform=transforms.Compose([ # TODO Pré-traitement à faire
            transforms.Resize(256),                    #[2]
            transforms.CenterCrop(224), 
            transforms.Lambda(duplicateChannel),
            transforms.ToTensor(),
            transforms.Normalize(mu,sigma, inplace=True)
        ]))
    val_dataset = datasets.ImageFolder(path+'/test',
        transform=transforms.Compose([ # TODO Pré-traitement à faire
            transforms.Resize(256),                    #[2]
            transforms.CenterCrop(224),
            transforms.Lambda(duplicateChannel),
            transforms.ToTensor(),
            transforms.Normalize(mu,sigma, inplace=True)
        ]))

    train_loader = torch.utils.data.DataLoader(train_dataset,
                        batch_size=batch_size, shuffle=False, pin_memory=CUDA, num_workers=2)
    val_loader = torch.utils.data.DataLoader(val_dataset,
                        batch_size=batch_size, shuffle=False, pin_memory=CUDA, num_workers=2)

    return train_loader, val_loader


In [ ]:
def extract_features(data, model):
    # TODO init features matrices
    X = np.zeros((len(data),data.batch_size,4096)) # 4096 dimension de l'avant derniere couche de VGG16.
    y = np.zeros((len(data),data.batch_size))
    model.eval()
    with torch.no_grad():
        for i, (x, target) in enumerate(data):
            if i % PRINT_INTERVAL == 0:
                print('Batch {0:03d}/{1:03d}'.format(i, len(data)))
            if CUDA:
                x = x.cuda()
            
            features = model(x)
            try:
                X[i] = features.detach().numpy()
                y[i]=target.detach().numpy()
            except:
                print("on ignore l'itération : ",i)
            print("i=",i)
            
    return X, y

def reshape_no_batch(X,y):
    # Enleve les batch et reshape les données numpy. 
    X = X.reshape((X.shape[0]*X.shape[1],X.shape[2]))
    y = y.reshape((y.shape[0]*y.shape[1]))

    return X,y

def main(path="15SceneData", batch_size=8):
   
    # Paramètres en ligne de commande
    parser = argparse.ArgumentParser()
    parser.add_argument('--path', default='15SceneData', type=str, metavar='DIR', help='path to dataset')
    parser.add_argument('--batch-size', default=20, type=int, metavar='N', help='mini-batch size (default: 8)')
    parser.add_argument('--cuda', dest='cuda', action='store_true', help='activate GPU acceleration')
    
    args = parser.parse_args()
    if args.cuda:
        CUDA = True
        cudnn.benchmark = True

    #main(args)
    params = args
    print('Instanciation de VGG16')
    vgg16 = models.vgg16(pretrained=True)

    print('Instanciation de VGG16relu7')
    model = VGG16relu7(vgg16) # TODO À remplacer par un reseau tronché pour faire de la feature extraction

    model.eval()
    if CUDA: # si on fait du GPU, passage en CUDA
        model = model.cuda()

    # On récupère les données
    print('Récupération des données')
    train, test = get_dataset(params.batch_size, params.path)

    # Extraction des features
    print('Feature extraction')
    if not os.path.exists('numpy_object/X_train.npy') : 
        X_train, y_train = extract_features(train, model)
        np.save('numpy_object/X_train.npy', X_train)
        np.save('numpy_object/y_train.npy', y_train)
    else:
        print("Chargement des fichiers X_train.npy et y_train.npy")
        X_train = np.load('numpy_object/X_train.npy')
        y_train = np.load('numpy_object/y_train.npy')

    X_train,y_train = reshape_no_batch(X_train,y_train)

    if not os.path.exists('numpy_object/X_test.npy') : 
        X_test, y_test = extract_features(test, model)
        np.save('numpy_object/X_test.npy', X_test)
        np.save('numpy_object/y_test.npy', y_test)
    else:
        X_test = np.load('numpy_object/X_test.npy')
        y_test = np.load('numpy_object/y_test.npy')

    X_test,y_test = reshape_no_batch(X_test,y_test)
    

    # TODO Apprentissage et évaluation des SVM à faire
    print('Apprentissage des SVM')
    svm = LinearSVC(C=1.0)
    svm.fit(X_train,y_train)
    accuracy = svm.score(X_test,y_test)
    print("SVM accuraccy C=1 :",accuracy)

    liste_C = [1e-3, 1e-2, 1e-1, 1, 10, 100, 1000]
    accuracy = []
    for c in liste_C:

        svm = LinearSVC(C=c)
        svm.fit(X_train,y_train)
        accuracy.append(svm.score(X_test,y_test))
    
    plt.plot(liste_C,accuracy)
    plt.ylabel('SVM Accuracy')
    plt.xlabel('paramètre C')
    plt.title('Exploration du paramètre C du SVM.')
    plt.show()
    
    input("done")


In [ ]:
main("15SceneData", 8)